In [1]:
import pandas as pd
import os
import shutil

In [3]:
case = "PJM_braidwood"
min_year = 2020
max_year = 2021
end_year = 2049
data_type = "DA"
#case = "NYISO_Ginna"
dest_folder = os.path.join("electricity_markets/",case)
original_data_folder = os.path.join(dest_folder,"original_data/")
original_files = os.listdir(original_data_folder)
use_case_folder_data = os.path.join("..","data")
use_case_folder_data
print(os.path.join(use_case_folder_data,dest_folder))

../data/electricity_markets/PJM_braidwood


In [4]:

def cleanup_data(df):
  todrop = []
  for c in df.columns:
    if "LBMP" in c:
      df.rename(columns={c:'price'}, inplace=True)
    elif "price"==c:
      pass
    else:
      todrop.append(c)
  df.drop(columns=todrop, axis=1, inplace=True)
  df['hour'] = [i for i in range(1,1+len(df))]
  # Remove additional day if necessary
  index_todrop= df[ df['hour'] >= 8761 ].index
  df.drop(index_todrop, inplace = True)
  return df

In [5]:
files_list = []
for f in original_files:
  if f.endswith('csv'):
    fsplit = f.split("_")
    year = int(fsplit[0])
    if min_year <= year and max_year >= year:
      present_file = os.path.abspath(os.path.join(original_data_folder,f))
      df = pd.read_csv(present_file)
      df = cleanup_data(df)
      # Move write cleaned-up data to destination folder
      clean_file = os.path.join(use_case_folder_data, dest_folder,f)
      files_list.append(clean_file)
      df.to_csv(clean_file, index=False)
      #print(df)
files_list.sort()
print(files_list)

['../data/electricity_markets/PJM_braidwood/2020_Braidwood_DA.csv', '../data/electricity_markets/PJM_braidwood/2021_Braidwood_DA.csv']


Create the pointer file for ARMA in train folder

In [6]:
plant = case.split("_")[1:]
print(plant)
if len(plant)>1:
  plant = ('_').join(plant)
else:
  plant = plant[0]
print(plant)
pointer_file = os.path.join("../train/","ARMA_pointer_{}_{}_{}_{}.csv".format(plant,str(min_year),str(max_year),data_type))
pointer_file
pointer_df = pd.DataFrame()
pointer_df['scaling'] = [1 for i in range(min_year, end_year+1)]
pointer_df['Year'] = [i for i in range(min_year, end_year+1)]
pointer_df['filename'] = [ files_list[i%len(files_list)] for i in range(end_year+1-min_year)]
pointer_df.to_csv(pointer_file, index=False)

['braidwood']
braidwood
